In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
folder_path = os.path.dirname(os.getcwd()) + "/data/tourism/palau/"
file_path = folder_path + os.listdir(folder_path)[0]

In [3]:
palau = pd.read_excel(file_path, sheet_name="TabMth")
palau = palau.iloc[0:-1].reset_index().drop("index", axis=1)
palau.columns = palau.iloc[0]
palau = palau.iloc[1:].T
palau = palau.reset_index()
palau.columns = palau.iloc[0]
palau = (palau.rename({"Country Group": "Date"}, axis=1)
         .iloc[1:]
         .reset_index()
         .drop("index", axis=1))

In [5]:
# Change the data format
palau["Date"] = pd.to_datetime(palau["Date"])
for col in palau.columns[~palau.columns.isin(["Date"])]:
    palau[col] = palau[col].astype(float)


palau.to_csv(folder_path + "palau_monthly_visitor.csv",
             encoding="utf-8")
palau.head(5)

,Date,JAPAN,SOUTH KOREA,TAIWAN,CHINA,USA/CANADA,EUROPE,OTHERS,Total
0,2007-06-01,856.0,1291.0,3245.0,86.0,669.0,99.0,463.0,6709.0
1,2007-07-01,2119.0,1366.0,3269.0,33.0,653.0,144.0,437.0,8021.0
2,2007-08-01,3476.0,1354.0,3046.0,46.0,580.0,256.0,438.0,9196.0
3,2007-09-01,3022.0,910.0,2497.0,61.0,559.0,145.0,401.0,7595.0
4,2007-10-01,1807.0,1082.0,2298.0,49.0,774.0,390.0,395.0,6795.0


In [6]:
from bokeh.palettes import Category20
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, HoverTool, Legend


output_file('palau_month_by_country.html')
p = figure(height=600, width=1000,
           title="Number of Passengers Per Month",
           x_axis_type="datetime",
           x_axis_label="Date",
           y_axis_label="Number of Passengers Per Month",
           tooltips=[("Country", "$name"),
                     ("Passenger per month", "@$name")])

countries = palau.columns[~palau.columns.isin(["Date", "Total"])].to_list()
p.add_layout(Legend(), 'right')
source = ColumnDataSource(palau)

for country, color in zip(countries, Category20[12]):
    # have to use different colnames for y-coords so tooltip can refer to @$name
    p.line('Date', country, source=source, name=country,
           legend_label=country, color=color)

p.legend.label_text_font_size = '9pt'
p.legend.click_policy = "mute"
p.legend.location = "top_left"

show(p)